In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table
from dash.dependencies import Input, Output


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient
import base64

from animalShelter import AnimalShelter



###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "newPass"
shelter = AnimalShelter(username, password)

df = pd.DataFrame.from_records(shelter.read({}))


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#Grazioso Salvare’s logo
image_filename = 'Grazioso_Salvare_Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Layout
app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('Cameron Michaud SNHU CS-340 Dashboard'))), # Identifier
    html.Hr(),
    html.Div(className = 'Filters',
            style = {'display' : 'flex'},
            children = [
                # Filter buttons
                html.Button(id = 'resetFilter', n_clicks_timestamp = 0, children = 'All'),                                     # Reset button
                html.Button(id = 'waterRescueFilter', n_clicks_timestamp = 0, children = 'Water Rescue'),                      # Water Rescue button
                html.Button(id = 'mountainRescueFilter', n_clicks_timestamp = 0, children = 'Mountain & Wilderness Rescue'),   # Mountain Rescue Button
                html.Button(id = 'disasterRescueFilter', n_clicks_timestamp = 0, children = 'Individual & Disaster Tracking'), # Individual Rescue button 
            ]),
    # Data Table
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        # Datatable preferences
        data=df.to_dict('records'),
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = "single",
        row_deletable = False,
        selected_columns = [],
        selected_rows = [],
        page_action = "native",
        page_current = 0,
        page_size = 10,

    ),
    html.Br(),
     html.Hr(),
    
    # Lower Element Alignments
     html.Div( className = 'row',
         style = {'display':'flex'},
         children = [
             html.Div( # Pie Chart
                 id = 'graph-id',
                 className = 'col s12 m6'),
             
             html.Div( # Map
                 id='map-id',
                 className='col s12 m6',),
             
             html.Div ( # Logo
                 html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode())),
             )
         ])
            
])

#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

#############################################
# Filter Button Callbacks
#############################################
@app.callback(
    Output('datatable-id', "data"),
     [Input('resetFilter','n_clicks_timestamp'),
     Input('waterRescueFilter', 'n_clicks_timestamp'), 
     Input('mountainRescueFilter', 'n_clicks_timestamp'),
     Input('disasterRescueFilter', 'n_clicks_timestamp')]
)
def on_click(reset, water, mountain, disaster): # Most recent filter selected is used, otherwise default order is shown
    # Reset is pressed
    if (int(reset) > int(water) and int(reset) > int(mountain) and int(reset) > int(disaster)):
        df = pd.DataFrame.from_records(shelter.read({}))
    # Water is pressed
    elif (int(water) > int(mountain) and int(water) > int(reset) and int(water) > int(disaster)):
        df = pd.DataFrame.from_records(shelter.read({'$and': [{"breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]}},
                                                   {"sex_upon_outcome": {"$in": ["Intact Female"]}},
                                                   {"age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}}]}))
    # Mountain is pressed
    elif (int(mountain) > int(water) and int(mountain) > int(reset) and int(mountain) > int(disaster)):     
        df = pd.DataFrame.from_records(shelter.read({'$and': [{"breed": {"$in": ["German Shepard", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]}},
                                                   {"sex_upon_outcome": {"$in": ["Intact Male"]}},
                                                   {"age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}}]})) 
    # Disaster is pressed
    elif (int(disaster) > int(water) and int(disaster) > int(reset) and int(disaster) > int(mountain)):     
        df = pd.DataFrame.from_records(shelter.read({'$and': [{"breed": {"$in": ["German Shepard", "Doberman Pinscher", "Golden Retriever", "Bloodhound", "Rottweiler"]}},
                                                   {"sex_upon_outcome": {"$in": ["Intact Male"]}},
                                                   {"age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}}]})) 
    # Default Data
    else:
        df = pd.DataFrame.from_records(shelter.read({}))
                                               
    # Set data to selected button
    return df.to_dict('records')

#############################################
# Pie Chart
#############################################
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    # Get current updated data table
    dff = pd.DataFrame.from_dict(viewData)
    # Get the names of the breeds of the table, get the amount each breed occurs
    breeds = dff['breed'].value_counts().keys().tolist()
    count = dff['breed'].value_counts().tolist()
    # Return updated pie chart
    return [
        dcc.Graph(            
            figure = px.pie(
                data_frame = dff, 
                values = count, 
                names = breeds, 
                color_discrete_sequence = px.colors.sequential.RdBu,
                width = 800, 
                height = 500   
            )
        )
    ]

#############################################
# Map
#############################################
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
     Input('datatable-id', 'selected_rows')])
def update_map(viewData, rows):
    dff = pd.DataFrame.from_dict(viewData)
    # Initialize to row 0, if none selected also default to row 0 to keep map displayed
    if rows == []:
        rows = [0]
    # Austin TX is at [30.75,-97.48]
    for row in rows:
        return [
            dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
                dl.TileLayer(id="base-layer-id"),
                # Marker with tool tip and popup
                dl.Marker(position=[(dff.iloc[row, 13]), (dff.iloc[row, 14])], children=[ # Use selected row, display content from specifc columns
                    dl.Tooltip(dff.iloc[row,4]),
                    dl.Popup([
                        html.H1("Animal Name:"),
                        html.Hr(),
                        html.P(dff.iloc[row, 9]),
                        
                        html.H1("Breed:"),
                        html.Hr(),
                        html.P(dff.iloc[row, 4]),
                        
                        # FUTURE tooltip data can extend from here
                    ])
                ])
            ])
        ]

app

Connection Successful
